In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import io
import os
import corrLib as cl
import matplotlib
import myImageLib as mil
import sys
import time

In [15]:
def dxd(div_folder, dcadv_folder):
    folder_list_dt = next(os.walk(dcadv_folder))[1]
    entries = ['adv_divv', 'adv_divcv', 'dc_divv', 'dc_divcv', 'vdc_divv', 'vdc_divcv']

    corr_lists = {}
    corr_values = {}

    for entry in entries:
        corr_lists[entry] = []    

    dtL = [] # Delta t
    for s in folder_list_dt:
        dt = int(s.split('=')[1])
        folder = os.path.join(dcadv_folder, s)
        l = cl.readdata(folder)
        l = l.loc[l.index > len(l)*0.9]
        count = 0

        for entry in entries:
            corr_values[entry] = 0
        for num, i in l.iterrows():
    #         print(i.Name)
            f, file = os.path.split(i.Dir)
            name = file.split('-')[0]
            n0 = int(name)
            n1 = n0 + 1
            divDir = os.path.join(div_folder, '{0:04d}-{1:04d}.csv'.format(n0, n1))
            divData = pd.read_csv(divDir)
            advData = pd.read_csv(i.Dir)

            for entry in entries:
                key1, key2 = entry.split('_')
                mat1, mat2 = advData[key1], divData[key2]
                corr_values[entry] += ((mat1 - mat1.mean())*(mat2 - mat2.mean())).mean() / mat1.std() / mat2.std()
            count += 1
        for entry in entries:
            corr_lists[entry].append(corr_values[entry] / count)
        dtL.append(dt)
    dxd_data = pd.DataFrame.from_dict(corr_lists).assign(dt=dtL)
    return dxd_data

In [17]:
num = range(0, 13)
for n in num:
    div_folder = r'E:\moreData\08062020\div2\{:02d}'.format(n)
    dcadv_folder = r'E:\moreData\08062020\dc_adv\{:02d}'.format(n)
    out_folder = r'E:\moreData\08062020\div_x_dcadv\{:02d}'.format(n)
    data = dxd(div_folder, dcadv_folder)
    data.to_csv(os.path.join(out_folder, 'divvXdcadv.csv'), index=False)

In [10]:
corr_values

{'adv_divv': -1.1521166479439338,
 'adv_divcv': -2.2150952249035,
 'dc_divv': 5.438751472905061,
 'dc_divcv': 5.504289844098644,
 'vdc_divv': -4.5039811770237765,
 'vdc_divcv': -5.786011484059659}

In [6]:
for s in folder_list_dt:
    print(s)

dt=60
